# ARCH-type models (package [`arch`](https://arch.readthedocs.io/en/latest/index.html))

In [ ]:
import numpy as np
import pandas as pd

from arch import arch_model

from arch.univariate import ARX, GARCH, ARCHInMean 

import pandas_datareader.data as web

# настройки визуализации
import matplotlib.pyplot as plt

# Не показывать Warnings
import warnings
warnings.simplefilter(action='ignore', category=Warning)

Basic ARCH-type models
* [`ARCH(p)`](https://arch.readthedocs.io/en/latest/univariate/generated/arch.univariate.ARCH.html#arch.univariate.ARCH)
* [`GARCH(p,0,q,power)`](https://arch.readthedocs.io/en/latest/univariate/generated/arch.univariate.GARCH.html#arch.univariate.GARCH)
* [`EGARCH(p,q)`](https://arch.readthedocs.io/en/latest/univariate/generated/arch.univariate.EGARCH.html#arch.univariate.EGARCH)
* [`APARCH(p,o,q,delta)`](https://arch.readthedocs.io/en/latest/univariate/generated/arch.univariate.APARCH.html#arch.univariate.APARCH)
* [`EWMAVariance(lambda)`](https://arch.readthedocs.io/en/latest/univariate/generated/arch.univariate.EWMAVariance.html#arch.univariate.EWMAVariance)

For `GARCH-in-mean` we use class [`ARCHInMean`](https://arch.readthedocs.io/en/latest/univariate/generated/arch.univariate.ARCHInMean.html#arch.univariate.ARCHInMean)

Let's load from [`FRED`](https://fred.stlouisfed.org/) weekly data on 3-Month Treasury Bill Secondary Market Rate (Symbol [`WTB3MS`](https://fred.stlouisfed.org/series/WTB3MS)) from 2000-01-01 to 2023-12-31 and let `y` be its first difference 

In [ ]:
rate = web.DataReader(name='WTB3MS', data_source='fred', start='2000-01-01', end='2023-12-31')
y = rate.diff().dropna()

y.plot()
plt.show()

## Fitting of AR-(*)ARCH of given order with `arch_model`


Let's fit AR(2)-GARCH(1,1)

Specification

\begin{align*}
	y_t&=\alpha_0+\phi_1y_{t-1}+\phi_2y_{t-2}+u_t \\ u_t&=\sigma_t \epsilon_t \\
	\sigma^\lambda_t &=\omega+\alpha|\epsilon_{t-1}|^\lambda+\beta\sigma^\lambda_{t-1}
\end{align*}

where $\lambda=2$

In [ ]:
am = arch_model(y, mean='ARX', lags=2, vol='GARCH', p=1, q=1, power=2)

res = am.fit()

res.summary()

In [ ]:
res.plot(annualize='W')
plt.show()

In [ ]:
res.hedgehog_plot(plot_type='volatility')
plt.show()

In [ ]:
res.hedgehog_plot(plot_type='mean')
plt.show()

In [ ]:
res.arch_lm_test(lags=5)

## Fitting of AR-(*)ARCH of given order

Let's fit AR(1)-GARCH(1,1,1)

Model's specification

\begin{align*}
	y_t&=\alpha_0+\phi_1y_{t-1}+u_t \\ u_t&=\sigma_t \epsilon_t \\
	\sigma^\lambda_t &=\omega+\alpha|\epsilon_{t-1}|^\lambda+\gamma|\epsilon_{t-1}|^\lambda I(\epsilon_{t-1}<0)
	 +\beta\sigma^\lambda_{t-1}
\end{align*}

where $\lambda=2$

In [ ]:
ar = ARX(y, lags=1)
ar.volatility = GARCH(p=1, o=1, q=1, power=2)

res = ar.fit()

res.summary()

## Forecasting

Let's evaluate 5-periods forecasts for AR(1)-GARCH(1,1)

Model's specification

\begin{align*}
	y_t&=\alpha_0+\phi y_{t-1}+u_t \\ u_t&=\sigma_t \epsilon_t \\
	\sigma^\lambda_t &=\omega+\alpha |\epsilon_{t-1}|^\lambda+\beta\sigma^\lambda_{t-1}
\end{align*}

where $\lambda=2$

In [ ]:
am = arch_model(y, mean='ARX', lags=1, vol='GARCH', p=1, q=1, o=0)

res = am.fit()

y_forecasts = res.forecast(horizon=5)

# forecasts in mean
y_forecasts.mean

In [ ]:
y_forecasts.residual_variance

In [ ]:
# forecasts of volatility
y_forecasts.variance

In [ ]:
am = arch_model(y, mean='ARX', lags=1, vol='EGARCH', p=1, q=1, o=0)

res = am.fit()

y_forecasts = res.forecast(horizon=1)

# forecasts in mean
y_forecasts.mean

In [ ]:
# forecasts of volatility
y_forecasts.variance

## Fitting GARCH-in-Mean

Model's specification

\begin{align*}
	y_t&=\alpha_0++\kappa\log \sigma^2_t+\phi y_{t-1}+u_t \\ u_t&=\sigma_t \epsilon_t \\
	\sigma^\lambda_t &=\omega+\alpha |\epsilon_{t-1}|^\lambda+\beta\sigma^\lambda_{t-1}
\end{align*}

where $\lambda=2$

In [ ]:
gim = ARCHInMean(y, lags=1, volatility=GARCH(p=1,o=0, q=1), form='log')
res = gim.fit()

res.summary()